In [1]:
import torch
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import statistics

from torch import nn
from copy import deepcopy
from tqdm import tqdm

from bayes_opt import BayesianOptimization

In [2]:
# Use the following gym version.
# pip install gym==0.25.0
# pip install pygame

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# define global variable 
MAX_EP = 1
env = gym.make('CartPole-v1')

/Users/jinhyunpark/miniforge3/envs/tilda/lib/python3.11/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/jinhyunpark/miniforge3/envs/tilda/lib/python3.11/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
# Define network architecture
class Network(nn.Module):
    def __init__(self, env, learning_rate=0.01):
        super().__init__()

        in_features = int(np.prod(env.observation_space.shape))
        self.net = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.Tanh(),
            nn.Linear(64, env.action_space.n)
        )
        
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=learning_rate)

    def forward(self, x):
        return self.net(x)

    def act(self, state):
        state_t = torch.as_tensor(state, dtype=torch.float32)
        q_values = self.forward(state_t.unsqueeze(0))                           # 'q_values' outputs two values (left or right)
        max_q_index = torch.argmax(q_values, dim=1)[0]                          # find an index that corresponds to the maximum value  
        action = max_q_index.detach().item()                                    # 0 or 1
        return action                                                           # 0 or 1

In [4]:
def calculate_fitness(network, env, num_episodes=MAX_EP):
    total_rewards = 0
    for _ in range(num_episodes):
        reward, _ = run_episode(network, env)
        total_rewards += reward
    avg_reward = total_rewards / num_episodes
    return avg_reward

In [5]:
def run_episode(network, env):
    state = env.reset()
    total_reward = 0.0
    log_probs = []  # Store log probabilities of actions
    done = False
    while not done:
        state_t = torch.as_tensor(state, dtype=torch.float32)
        q_values = network(state_t.unsqueeze(0))
        action_probs = nn.functional.softmax(q_values, dim=1)
        action_dist = torch.distributions.Categorical(action_probs)
        action = action_dist.sample()
        log_prob = action_dist.log_prob(action)
        log_probs.append(log_prob)
        state, reward, done, _ = env.step(action.item())
        total_reward += reward
    return total_reward, log_probs

In [6]:
def prepare_mutation(network, env, num_episodes=MAX_EP):
    total_rewards = 0
    all_grads = []  # List of list of gradients
    for _ in range(num_episodes):
        reward, log_probs = run_episode(network, env)
        total_rewards += reward
        loss = -reward * torch.stack(log_probs).sum()
        loss.backward()  # Calculate gradients
        network.optimizer.step()

        grads = []  # List of gradients for this episode
        # Store gradients and zero them
        for param in network.parameters():
            grads.append(param.grad.clone())
            param.grad = None
        
        all_grads.append(grads)  # Append the list of gradients for this episode to the overall list
    
    avg_reward = total_rewards / num_episodes
    
    return avg_reward, all_grads  # Return list of list of gradients

In [7]:
def mutate_and_tournament(population, tournament_size, mutation_rate, mutation_strength):
    
    # Select individuals for the tournament
    individuals = random.sample(population, tournament_size)
    # Calculate fitness for each individual
    fitnesses = [calculate_fitness(individual, env) for individual in individuals]
    # Select the best individual
    parent = individuals[np.argmax(fitnesses)]
    
    # Create offspring by deep copying the parent
    offspring = deepcopy(parent)
    
    # Calculate fitness and gradients for the offspring
    _, all_grads = prepare_mutation(offspring, env)
    
    grads = []
    # Average gradients over episodes 
    for grad in zip(*all_grads):
        grads.append(sum(grad)/len(grad))
    
    # Apply mutation
    with torch.no_grad():
        for param, grad in zip(offspring.parameters(), grads):
            if (grad is not None) and (random.random() < mutation_rate):
                # print("mutation activated")
                delta = torch.randn_like(param)
                grad_sum = torch.sum(grad)
                if grad_sum != 0:
                    param.add_(mutation_strength * delta * grad / grad_sum)
    
    # Return the mutated offspring
    return offspring

In [8]:
# Define genetic algorithm
def main(POPULATION_SIZE, GENERATIONS, ELITISM, TOURNAMENT_SIZE, MUTATION_STRENGTH, MUTATION_RATE):
    
    FITNESS_HISTORY = list()
    FITNESS_STDERROR_HISTORY = list()
    
    # Create initial population
    population = [Network(env) for _ in range(POPULATION_SIZE)]

    for generation in range(1, GENERATIONS + 1):

        # Calculate fitness for each network
        fitnesses = [calculate_fitness(network, env) for network in tqdm(population, desc="Calculating fitnesses")]
        
        # average fitness 
        avg_fitness = np.average(fitnesses)
        max_fitness = np.max(fitnesses)
        min_fitness = np.min(fitnesses)
        FITNESS_HISTORY.append([avg_fitness, max_fitness, min_fitness])
        
        # std error
        standard_deviation = statistics.stdev(fitnesses)
        standard_error = standard_deviation / (POPULATION_SIZE ** 0.5)
        FITNESS_STDERROR_HISTORY.append(standard_error)

        print(f"[Generation: {generation}] \n Average Fitness: {avg_fitness} \n Best Fitness: {max_fitness} \n Worst Fitness: {min_fitness} \n Standard Error: {standard_error}")
        
        # Sort population by fitness
        population = [x for _, x in sorted(zip(fitnesses, population), key=lambda pair: pair[0], reverse=True)]
        
        # Select the best networks to pass their genes to the next generation
        survivors = population[:ELITISM]
        
        # Create the next generation
        next_population = survivors  # Start with the survivors
        
        num_individuals_to_add = POPULATION_SIZE - len(next_population)
        # Add offspring by tournament selection and mutation
        for _ in tqdm(range(num_individuals_to_add), desc="Generating Offspring"):
            offspring = mutate_and_tournament(population, TOURNAMENT_SIZE, MUTATION_RATE, MUTATION_STRENGTH)
            next_population.append(offspring)

        # The next generation becomes the current population
        population = next_population

    return population, FITNESS_HISTORY, FITNESS_STDERROR_HISTORY

### Version Control

In [ ]:
# Version 1:
# Run the genetic algorithm
population, history, history_std = main(POPULATION_SIZE=200, 
                            GENERATIONS=100, 
                            ELITISM=100, 
                            TOURNAMENT_SIZE=3, 
                            MUTATION_STRENGTH=1, 
                            MUTATION_RATE=0.01)

Calculating fitnesses:   0%|                                                                                                                                   | 0/200 [00:00<?, ?it/s]/Users/jinhyunpark/miniforge3/envs/tilda/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
Calculating fitnesses: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 582.44it/s]


[Generation: 1] 
 Average Fitness: 21.49 
 Best Fitness: 97.0 
 Worst Fitness: 9.0 
 Standard Error: 0.8837730330064087


Calculating fitnesses: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 537.91it/s]


[Generation: 2] 
 Average Fitness: 23.195 
 Best Fitness: 87.0 
 Worst Fitness: 9.0 
 Standard Error: 1.0060045231024124


Calculating fitnesses: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 605.85it/s]


...

Calculating fitnesses: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:05<00:00, 38.18it/s]


[Generation: 52] 
 Average Fitness: 320.935 
 Best Fitness: 500.0 
 Worst Fitness: 93.0 
 Standard Error: 9.627598722464704


Calculating fitnesses: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:05<00:00, 38.14it/s]


[Generation: 53] 
 Average Fitness: 325.205 
 Best Fitness: 500.0 
 Worst Fitness: 88.0 
 Standard Error: 9.55883473454856


Generating Offspring:  56%|████████████████████████████████████████████████████████████████████▉                                                      | 56/100 [00:06<00:04,  9.34it/s]

In [ ]:
plt.figure(figsize=(20, 15))
plt.plot(np.arange(100), np.array(history)[:,0], marker='o', linestyle='-', label='Average Fitness')
plt.plot(np.arange(100), np.array(history)[:,1], marker='^', linestyle='-', label='Max Fitness')
plt.plot(np.arange(100), np.array(history)[:,2], marker='s', linestyle='-', label='Min Fitness')
plt.fill_between(np.arange(100), 0, np.array(history)[:,0] + np.array(history)[:,1],
                 alpha=0.2, color='blue', label='Standard Error')

plt.xlabel('Generations')
plt.ylabel('Fitness')
plt.title('Fitness History')
plt.grid()
plt.legend()
plt.show()